In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry

ModuleNotFoundError: No module named 'ngsolve'

## 1) Construction du maillage

In [ ]:
geo = SplineGeometry()

geo.AddRectangle(p1=(-3,-3),p2=(3,3),bc="rectangle",leftdomain=1,rightdomain=0,
                 bcs = ("right", "top", "left", "bot"))
geo.AddRectangle(p1=(-0.6,-0.2), p2=(-0.5,0.2),bc="rectangle",leftdomain=2,rightdomain=1)
geo.AddRectangle(p1=(-0.3,-0.2),p2=(-0.2,0.2),bc="rectangle",leftdomain=3,rightdomain=1)
geo.AddRectangle(p1=(0.2,-0.2),p2=(0.3,0.2),bc="rectangle",leftdomain=4,rightdomain=1)
geo.AddRectangle(p1=(0.5,-0.2),p2=(0.6,0.2),bc="rectangle",leftdomain=5,rightdomain=1)

geo.SetMaterial (1, "zone_optim")
geo.SetMaterial (2, "primaire_p")
geo.SetMaterial (3, "primaire_m")
geo.SetMaterial (4, "secondaire_p")
geo.SetMaterial (5, "secondaire_m")

#geo.SetDomainMaxH(1, 0.05)
ngmesh = geo.GenerateMesh(maxh=0.1)
mesh = Mesh(ngmesh)
Draw (mesh)
mesh.GetBoundaries()

In [ ]:
J= 1e6;

fespace_H1 = H1(mesh, order=1, dirichlet="right|top|left|bot")

a = fespace_H1.TrialFunction()
v = fespace_H1.TestFunction()

f = LinearForm(fespace_H1)
f += v*J*dx("primaire_p") - v*J*dx("primaire_m")

f.Assemble()

nu0 = 1/(4e-7*3.14)
FB = BilinearForm(fespace_H1)
FB += grad(a)*nu0*grad(v)*dx

FB.Assemble()

In [ ]:
solution = GridFunction(fespace_H1)  # solution 
solution.vec.data = FB.mat.Inverse(fespace_H1.FreeDofs()) * f.vec
Draw (solution)

In [ ]:
derivee = solution * nu0* solution;

In [ ]:
Draw (derivee,mesh)